In [ ]:
!7z x new.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1083189 bytes (1058 KiB)

Extracting archive: new.zip
--
Path = new.zip
Type = zip
Physical Size = 1083189
Comment = a0f1824bbf79f6b56df28ace7d143bd5004cf8c2

  0%    Everything is Ok

Folders: 1
Files: 8
Size:       3467546
Compressed: 1083189


In [ ]:
df = pd.read_csv("ADHD.csv")
df

,Unnamed: 0,text,content,wording,prompt_id,prompt_question,prompt_title,prompt_text,student_id
0,0,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,ded6a14b4c27
1,1,"Hey, did yOU kow? Guess what? So the social go...",0.237703,0.500000,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,95c8b1d33694
2,2,"HEy, did you know? Here's a story: The Differe...",0.378200,0.542762,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,55918828da86
3,3,"Guess what? Hey, did you know? The pharaohs, t...",0.789650,0.859773,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,e70311747892
4,4,"iN ancienct Egpt, tEy could raed and wrtie. Th...",0.495394,0.500000,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,875c0d066910
...,...,...,...,...,...,...,...,...,...
195,195,"Did you know? HEY, did you konw? Imagine this ...",0.902170,0.924963,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,be6dddc74b14
196,196,third wave WAs developed over scuh A short per...,0.627952,0.500000,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,5f5d3ecc6779
197,197,"Hye, did you knOW? The Trd WAVe developd oevr ...",0.699641,0.500000,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,1dc0c8ae7019
198,198,"Imagine this - Hye, did you know? Here's a sto...",0.218599,0.500000,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,21c30b4863fd


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoTokenizer, AutoModel
#from transformers import DebertaTokenizer, DebertaModel
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline
from argparse import Namespace
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut

from tqdm import tqdm as tq
from tqdm import notebook

In [ ]:
import random
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
# prompt: store sum_train = df['student_id',	'prompt_id'	,'text',	'content',	'wording']

sum_train = df[['student_id', 'prompt_id', 'text', 'content', 'wording']]
sum_train.head(3)


,student_id,prompt_id,text,content,wording
0,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241
1,95c8b1d33694,3b9047,"Hey, did yOU kow? Guess what? So the social go...",0.237703,0.500000
2,55918828da86,3b9047,"HEy, did you know? Here's a story: The Differe...",0.378200,0.542762


In [ ]:
#sum_train=pd.read_csv('/content/CommonLit-Evaluate-Student-Summaries-main/summaries_train.csv')
sum_train = df['student_id',	'prompt_id'	,'text',	'content',	'wording']

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757
...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128


## Preprocessing

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Removing numbers
    text = re.sub(r'\d+', '', text)

    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenization
    tokens = word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Joining tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [ ]:
sum_train.prompt_id.nunique()

4

In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokenizer =AutoTokenizer.from_pretrained("microsoft/deberta-base")
sep=tokenizer.sep_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
import pickle
with open("bert_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [ ]:
tkn=tokenizer.tokenize('Hello, student 2023!')
tkn

['Hello', ',', 'Ġstudent', 'Ġ20', '23', '!']

In [ ]:
sum_train['len_text']=sum_train.text.apply(lambda x: len(tokenizer.tokenize(x)))
max_len=sum_train.len_text.max()
q_97=sum_train.len_text.quantile(q=0.97)
sum_train.head(3)

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-11-433fcd8e4a2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_train['len_text']=sum_train.text.apply(lambda x: len(tokenizer.tokenize(x)))


,student_id,prompt_id,text,content,wording,len_text
0,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,318
1,95c8b1d33694,3b9047,"Hey, did yOU kow? Guess what? So the social go...",0.237703,0.500000,90
2,55918828da86,3b9047,"HEy, did you know? Here's a story: The Differe...",0.378200,0.542762,102


In [ ]:
max_len, q_97

(581, 446.14000000000004)

In [ ]:
prompts_train=df[['prompt_id',	'prompt_question'	,'prompt_title'	,'prompt_text']]

In [ ]:
#prompts_train=pd.read_csv('/content/CommonLit-Evaluate-Student-Summaries-main/prompts_train.csv')
prompts_train['prompt_text_token']=prompts_train.prompt_text.apply(lambda x: tokenizer.tokenize(x)[:1200])
prompts_train['prompt_text_ids']=prompts_train.prompt_text_token.apply(lambda x: [1]+tokenizer.convert_tokens_to_ids(x)+[2])
#prompts_train['prompt_q_token']=prompts_train.prompt_question.apply(lambda x: tokenizer.tokenize(x)[:1200])
#prompts_train['prompt_q_ids']=prompts_train.prompt_q_token.apply(lambda x: [1]+tokenizer.convert_tokens_to_ids(x)+[2])
prompts_train

<ipython-input-14-91d5d713a521>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompts_train['prompt_text_token']=prompts_train.prompt_text.apply(lambda x: tokenizer.tokenize(x)[:1200])
<ipython-input-14-91d5d713a521>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompts_train['prompt_text_ids']=prompts_train.prompt_text_token.apply(lambda x: [1]+tokenizer.convert_tokens_to_ids(x)+[2])


,prompt_id,prompt_question,prompt_title,prompt_text,prompt_text_token,prompt_text_ids
0,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"[Egypt, ian, Ġsociety, Ġwas, Ġstructured, Ġlik...","[1, 37552, 811, 2313, 21, 16697, 101, 10, 3334..."
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"[Egypt, ian, Ġsociety, Ġwas, Ġstructured, Ġlik...","[1, 37552, 811, 2313, 21, 16697, 101, 10, 3334..."
2,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"[Egypt, ian, Ġsociety, Ġwas, Ġstructured, Ġlik...","[1, 37552, 811, 2313, 21, 16697, 101, 10, 3334..."
3,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"[Egypt, ian, Ġsociety, Ġwas, Ġstructured, Ġlik...","[1, 37552, 811, 2313, 21, 16697, 101, 10, 3334..."
4,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"[Egypt, ian, Ġsociety, Ġwas, Ġstructured, Ġlik...","[1, 37552, 811, 2313, 21, 16697, 101, 10, 3334..."
...,...,...,...,...,...,...
195,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,"[Background, Ġ, č, Ċ, The, ĠThird, ĠWave, Ġexp...","[1, 48277, 1437, 50121, 50118, 133, 7470, 2118..."
196,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,"[Background, Ġ, č, Ċ, The, ĠThird, ĠWave, Ġexp...","[1, 48277, 1437, 50121, 50118, 133, 7470, 2118..."
197,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,"[Background, Ġ, č, Ċ, The, ĠThird, ĠWave, Ġexp...","[1, 48277, 1437, 50121, 50118, 133, 7470, 2118..."
198,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,"[Background, Ġ, č, Ċ, The, ĠThird, ĠWave, Ġexp...","[1, 48277, 1437, 50121, 50118, 133, 7470, 2118..."


In [ ]:
prompts_train['len_prompt']=prompts_train.prompt_text.apply(lambda x: len(tokenizer.tokenize(x)))
prompts_train.len_prompt.max(), prompts_train.len_prompt.min()

<ipython-input-15-f368fce6cd77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompts_train['len_prompt']=prompts_train.prompt_text.apply(lambda x: len(tokenizer.tokenize(x)))


(1199, 700)

In [ ]:
model = AutoModel.from_pretrained("microsoft/deberta-base")

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
model

DebertaModel(
  (embeddings): DebertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=0)
    (LayerNorm): DebertaLayerNorm()
    (dropout): StableDropout()
  )
  (encoder): DebertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x DebertaLayer(
        (attention): DebertaAttention(
          (self): DisentangledSelfAttention(
            (in_proj): Linear(in_features=768, out_features=2304, bias=False)
            (pos_dropout): StableDropout()
            (pos_proj): Linear(in_features=768, out_features=768, bias=False)
            (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
            (dropout): StableDropout()
          )
          (output): DebertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): DebertaLayerNorm()
            (dropout): StableDropout()
          )
        )
        (intermediate): DebertaIntermediate(
          (dense): Linear(in_features=768, out_features=3

In [ ]:
with open("deberta_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
def make_sentence(x):
    #model.eval()
    model.eval
    text=torch.LongTensor(x).unsqueeze(dim=0)
    with torch.no_grad():
        #enc_layer, _=model(text)
        last_layer=model(text).last_hidden_state
        #poll_out=model(text).pooler_output
    #return torch.mean(enc_layer[11], 1).squeeze(dim=0)
    #return(poll_out.squeeze(dim=0))
    return torch.mean(last_layer, 1).squeeze(dim=0)
        #return model_seq(text).logits.squeeze()

In [ ]:
prompts_train['prompt_text_sent']=prompts_train['prompt_text_ids'].apply(make_sentence)
#prompts_train['prompt_q_sent']=prompts_train['prompt_q_ids'].apply(make_sentence)

<ipython-input-20-7e89d1cb3b1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompts_train['prompt_text_sent']=prompts_train['prompt_text_ids'].apply(make_sentence)


In [ ]:
train_content=pd.merge(sum_train, prompts_train, how='left', left_on='prompt_id', right_on='prompt_id')
train_content.isnull().sum()

student_id           0
prompt_id            0
text                 0
content              0
wording              0
len_text             0
prompt_question      0
prompt_title         0
prompt_text          0
prompt_text_token    0
prompt_text_ids      0
len_prompt           0
prompt_text_sent     0
dtype: int64

In [ ]:
train_content['len_ratio']=train_content.len_text/train_content.len_prompt

In [ ]:
train_c=train_content.drop(['prompt_text', 'prompt_title','prompt_question', 'prompt_text_ids',\
                           'prompt_text_token', 'len_text', 'len_prompt'], axis=1)
train_c.head(3)

,student_id,prompt_id,text,content,wording,prompt_text_sent,len_ratio
0,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.42915
1,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.42915
2,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.42915


In [ ]:
SPLIT_SEED=92
from sklearn.model_selection import train_test_split

#data_train, data_val = train_test_split(train_c, test_size=0.20, random_state=SPLIT_SEED)
prompt_list=list(prompts_train['prompt_id'])
data_train=train_c[(train_c.prompt_id==prompt_list[0]) | (train_c.prompt_id==prompt_list[1]) |\
                   (train_c.prompt_id==prompt_list[3])].copy()

data_val=train_c[(train_c.prompt_id==prompt_list[2])].copy()

data_train['split']='train'
data_val['split']='val'
data_with_split=pd.concat([data_train, data_val], ignore_index=True)
data_with_split

,student_id,prompt_id,text,content,wording,prompt_text_sent,len_ratio,split
0,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.429150,train
1,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.429150,train
2,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.429150,train
3,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.429150,train
4,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.429150,train
...,...,...,...,...,...,...,...,...
4995,679dd420c92d,3b9047,"animals, kept canas and resrevoirs in good ord...",0.729015,0.593298,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.462888,val
4996,679dd420c92d,3b9047,"animals, kept canas and resrevoirs in good ord...",0.729015,0.593298,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.462888,val
4997,679dd420c92d,3b9047,"animals, kept canas and resrevoirs in good ord...",0.729015,0.593298,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.462888,val
4998,679dd420c92d,3b9047,"animals, kept canas and resrevoirs in good ord...",0.729015,0.593298,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.462888,val


In [ ]:
class TextDataset(Dataset):
    def __init__(self, text_df, max_seq_length):

        self.text_df = text_df

        self._max_seq_length = max_seq_length

        self.train_df = self.text_df[self.text_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.text_df[self.text_df.split=='val']
        self.validation_size = len(self.val_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size)}

        self.set_split('train')


    def set_split(self, split="train"):
        self._data_split = split
        self._data_df, self._data_size = self._lookup_dict[split]

    def __len__(self):
        return self._data_size

    def __getitem__(self, index):

        row = self._data_df.iloc[index]
        text=row['text']
        tokens=tokenizer.tokenize(text)
        text_index=[1]+tokenizer.convert_tokens_to_ids(tokens)+[2]
        token_index=text_index

        if len(token_index)<self._max_seq_length:
            pad=[0]*(self._max_seq_length-len(token_index))
            token_index=token_index+pad

        else:
            token_index=token_index[:self._max_seq_length]

        data_vector = torch.LongTensor(token_index)

        target = row[['content', 'wording']]

        return {'x_data': data_vector,
                'attention_mask': (data_vector!=0).long(),
                'content_vector': row['prompt_text_sent'],
                'y_target': torch.squeeze(torch.FloatTensor([target])),
                'len_ratio': row['len_ratio']}

    def get_num_batches(self, batch_size):

        return len(self) // batch_size

In [ ]:
data=TextDataset(data_with_split, 350)

In [ ]:
t=data.__getitem__(0)
#t

In [ ]:
t['x_data'].shape, t['content_vector'].shape, t['y_target'].shape, t['attention_mask'].shape

(torch.Size([350]), torch.Size([768]), torch.Size([2]), torch.Size([350]))

In [ ]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)

        yield out_data_dict

In [ ]:
gen=generate_batches(data, 3)

In [ ]:
g=next(gen)
g['x_data'].shape, g['y_target'].shape, g['len_ratio'].shape

(torch.Size([3, 350]), torch.Size([3, 2]), torch.Size([3]))

In [ ]:
class BertForSequenceRegression(nn.Module):

    def __init__(self, num_marks=2):
        super(BertForSequenceRegression, self).__init__()
        self.num_marks = num_marks
        with open("/content/deberta_model.pkl", "rb") as f:
            self.bert = pickle.load(f)

        self.hidden_1=nn.Linear(2*config.hidden_size, 2*config.hidden_size)
        self.notline_1=nn.ReLU()
        self.dropout_1 = nn.Dropout(config.hidden_dropout_prob)
        self.hidden_2=nn.Linear(2*config.hidden_size, config.hidden_size)
        self.notline_2=nn.ReLU()

        self.dropout_2 = nn.Dropout(config.hidden_dropout_prob)
        self.hidden = nn.Linear(config.hidden_size, 128)
        self.regres = nn.Linear(128, num_marks)

    def forward(self, input_ids, content_vector,\
                token_type_ids=None, attention_mask=None, labels=None):
        output_bert = self.bert(input_ids, token_type_ids, attention_mask).last_hidden_state.mean(1)
        h_concat=torch.cat((content_vector,output_bert), dim=1)
        hidden_vec_1=self.hidden_1(h_concat)
        hidden_drop_1=self.dropout_1(hidden_vec_1)
        hidden_vecn_1=self.notline_1(hidden_drop_1)
        hidden_vec_2=self.hidden_2(hidden_vecn_1)
        hidden_drop_2=self.dropout_2(hidden_vec_2)
        hidden_vecn_2=self.notline_2(hidden_drop_2)
        hidden=self.hidden(hidden_vecn_2)
        marks = self.regres(hidden)

        return marks

In [ ]:
config=Namespace(
hidden_dropout_prob=0.05,
hidden_size=768
)

In [ ]:
model=BertForSequenceRegression()
model

BertForSequenceRegression(
  (bert): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermedia

In [ ]:
from sklearn.metrics import mean_squared_error as RMSE

In [ ]:
args = Namespace(
    # Training hyper parameter
    num_epochs=50,
    learning_rate=1.2e-5,
    batch_size=4,
    seed=99,
    cuda=True
    )

In [ ]:
def make_train_state(args):
    return {
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': []
            }

In [ ]:
make_train_state(args)

{'learning_rate': 1.2e-05,
 'epoch_index': 0,
 'train_loss': [],
 'train_acc': [],
 'val_loss': [],
 'val_acc': []}

In [ ]:
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")

print("Using CUDA: {}".format(args.cuda))

Using CUDA: True


In [ ]:

train_c.head(1)

,student_id,prompt_id,text,content,wording,prompt_text_sent,len_ratio
0,ded6a14b4c27,3b9047,"Hey, did you knwo? At the top of the socal lad...",0.863521,0.560241,"[tensor(0.0025), tensor(0.3075), tensor(-0.016...",0.42915


In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()

In [ ]:
groups = train_c['prompt_id']
for i, (train_index, test_index) in enumerate(logo.split(train_c, groups=groups)):
    X_train = train_c.iloc[train_index].copy()
    X_train['split']='train'
    X_val = train_c.iloc[test_index].copy()
    X_val['split']='val'
    X=pd.concat([X_train, X_val], ignore_index=True)
    print(X[X.split=='train'].prompt_id.value_counts())
    print(X[X.split=='val'].prompt_id.value_counts())

3b9047    2500
ebad26    2500
814d6b    2500
Name: prompt_id, dtype: int64
39c16e    2500
Name: prompt_id, dtype: int64
ebad26    2500
39c16e    2500
814d6b    2500
Name: prompt_id, dtype: int64
3b9047    2500
Name: prompt_id, dtype: int64
3b9047    2500
ebad26    2500
39c16e    2500
Name: prompt_id, dtype: int64
814d6b    2500
Name: prompt_id, dtype: int64
3b9047    2500
39c16e    2500
814d6b    2500
Name: prompt_id, dtype: int64
ebad26    2500
Name: prompt_id, dtype: int64


In [ ]:
loss_func = nn.MSELoss()
groups = train_c['prompt_id']
logo = LeaveOneGroupOut()

for i, (train_index, test_index) in enumerate(logo.split(train_c, groups=groups)):
    min_val_loss = np.inf
    train_state = make_train_state(args)
    X_train = train_c.iloc[train_index].copy()
    X_train['split']='train'
    X_val = train_c.iloc[test_index].copy()
    X_val['split']='val'
    X=pd.concat([X_train, X_val], ignore_index=True)

    data=TextDataset(X, 260)


    running_loss = 0.0
    running_acc = 0.0
    patience = 0

    #regr = BertForSequenceRegression().to(args.device)
    regr = BertForSequenceRegression().to(args.device)

    lrlast = .001
    lrmain = args.learning_rate
    optimizer = optim.Adam(
    [
        {"params":regr.bert.parameters(),"lr": lrmain},
        {"params":regr.regres.parameters(), "lr": lrlast},

   ])


    print(f'-------------------- Fold {i+1} --------------------')
    for epoch in range(args.num_epochs):
        regr.train()
        data.set_split('train')
        batch_generator = generate_batches(data,
                                           batch_size=args.batch_size,
                                           device=args.device)

        running_loss = 0.0
        running_acc = 0.0
        for batch_index, batch_dict in enumerate(batch_generator):
            n_batch=batch_index
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients

            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = regr(batch_dict['x_data'], batch_dict['content_vector'], \
                          attention_mask=batch_dict['attention_mask'])

            # step 3. compute the loss

            loss = loss_func(y_pred, batch_dict['y_target'].float())

            #running_loss += (loss.item() - running_loss) / (batch_index + 1)
            running_loss += loss.item()

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the RMSE
            acc_t = RMSE(batch_dict['y_target'].to('cpu').numpy(), y_pred.detach().to('cpu').numpy(), squared=False)
            #running_acc += (acc_t - running_acc) / (batch_index + 1)
            running_acc += acc_t

        avg_loss=running_loss/(n_batch+1)

        train_state['train_loss'].append(running_loss/(n_batch+1))
        train_state['train_acc'].append(running_acc/(n_batch+1))

        # Validation loop
        data.set_split('val')
        batch_generator = generate_batches(data,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        regr.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            n_batch=batch_index
            # compute the output
            y_pred = regr(batch_dict['x_data'], batch_dict['content_vector'],\
                          attention_mask=batch_dict['attention_mask'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            #running_loss += (loss.item() - running_loss) / (batch_index + 1)
            running_loss += loss.item()
            # compute the accuracy
            acc_t = RMSE(batch_dict['y_target'].to('cpu').numpy(), y_pred.detach().to('cpu').numpy(), squared=False)
            #running_acc += (acc_t - running_acc) / (batch_index + 1)
            running_acc += acc_t

        avg_acc = running_acc/(n_batch+1)
        avg_val_loss = running_loss/(n_batch+1)

        train_state['val_loss'].append(running_loss/(n_batch+1))
        train_state['val_acc'].append(running_acc/(n_batch+1))

        print(f'Epoch {epoch+1} Loss: {avg_loss:.3f} SCORE:{avg_acc:.3}')

        #if (avg_val_loss < min_val_loss) and (epoch!=0):
        if (avg_val_loss < min_val_loss):
            patience = 0
            min_val_loss = avg_val_loss
            model_name='adhd_deb_model' +'_'+ str(i)+'.pt'
            torch.save(regr.state_dict(), model_name)
            print(f'saving model with score: {avg_acc:.3f}')

        patience += 1
        if patience >=3 :
            print(f'Early Stopping trigerred on epoch: {epoch+1}')
            break


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


-------------------- Fold 1 --------------------
Epoch 1 Loss: 0.007 SCORE:0.117
saving model with score: 0.117
Epoch 2 Loss: 0.001 SCORE:0.102
saving model with score: 0.102
Epoch 3 Loss: 0.001 SCORE:0.106
Epoch 4 Loss: 0.000 SCORE:0.107
Early Stopping trigerred on epoch: 4
-------------------- Fold 2 --------------------
Epoch 1 Loss: 0.006 SCORE:0.141
saving model with score: 0.141
Epoch 2 Loss: 0.001 SCORE:0.118
saving model with score: 0.118
Epoch 3 Loss: 0.001 SCORE:0.115
saving model with score: 0.115
Epoch 4 Loss: 0.001 SCORE:0.115
Epoch 5 Loss: 0.000 SCORE:0.122
Early Stopping trigerred on epoch: 5
-------------------- Fold 3 --------------------
Epoch 1 Loss: 0.005 SCORE:0.119
saving model with score: 0.119
Epoch 2 Loss: 0.001 SCORE:0.125
Epoch 3 Loss: 0.000 SCORE:0.13
Early Stopping trigerred on epoch: 3
-------------------- Fold 4 --------------------
Epoch 1 Loss: 0.006 SCORE:0.135
saving model with score: 0.135
Epoch 2 Loss: 0.001 SCORE:0.14
Epoch 3 Loss: 0.001 SCORE:0.11

In [ ]:
y_pred.shape

torch.Size([4, 2])

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn


# Define the device to be CUDA if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# Load the trained model weights
i = 3 # Assuming you have a variable 'i' defined elsewhere
model_name = 'deb_model' + '_' + str(i) + '.pt'
regr.load_state_dict(torch.load(model_name, map_location=device))

# Sample text
text = 'Good Morning '

# Tokenize the text
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize(text)
text_index = [tokenizer.cls_token_id] + tokenizer.convert_tokens_to_ids(tokens) + [tokenizer.sep_token_id]

# Pad or truncate the token index
max_length = 260
if len(text_index) < max_length:
    pad = [tokenizer.pad_token_id] * (max_length - len(text_index))
    token_index = text_index + pad
else:
    token_index = text_index[:max_length]

# Convert token index to tensor
data_vector = torch.LongTensor(token_index).unsqueeze(dim=0).to(device)

# Generate content vector (random for illustration)
content_vector = torch.randn(1, 768).to(device)

# Generate attention mask
attention_mask = (data_vector != 0).long()

# Perform prediction
with torch.no_grad():
    y_pred = regr(data_vector, content_vector, attention_mask=attention_mask)

print(y_pred)

import torch.nn.functional as F

# Apply sigmoid function to convert logits to probabilities
probs = F.sigmoid(y_pred)

# Display the probabilities
print(probs)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tensor([[-0.8473, -1.0522]], device='cuda:0')
tensor([[0.3000, 0.2588]], device='cuda:0')


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn


# Define the device to be CUDA if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize lists to store the predictions for content and wording scores from each model
all_content_scores = []
all_wording_scores = []

# Iterate over each model
for i in range(1, 4):
    # Load the trained model
    model_name = 'adhd_deb_model_' + str(i) + '.pt'
    model = BertForSequenceRegression().to(device)
    model.load_state_dict(torch.load(model_name, map_location=device))
    model.eval()

    # Sample text
    text = 'Good Morning and my name is ali'

    # Tokenize the text
    tokens = tokenizer.tokenize(text)
    token_index = [tokenizer.cls_token_id] + tokenizer.convert_tokens_to_ids(tokens) + [tokenizer.sep_token_id]

    # Pad or truncate the token index
    max_length = 260
    if len(token_index) < max_length:
        pad = [tokenizer.pad_token_id] * (max_length - len(token_index))
        token_index = token_index + pad
    else:
        token_index = token_index[:max_length]

    # Convert token index to tensor
    data_vector = torch.LongTensor(token_index).unsqueeze(dim=0).to(device)

    # Generate content vector (random for illustration)
    content_vector = torch.randn(1, 768).to(device)

    # Generate attention mask
    attention_mask = (data_vector != 0).long()

    # Perform prediction
    with torch.no_grad():
        scores = model(data_vector, content_vector, attention_mask=attention_mask)

    # Parse the output tensor to extract content and wording scores
    content_score, wording_score = scores[:, 0], scores[:, 1]

    # Append the predictions to the respective lists
    all_content_scores.append(content_score.item())
    all_wording_scores.append(wording_score.item())

# Find the index of the model with the best content score
best_content_index = all_content_scores.index(max(all_content_scores))

# Find the index of the model with the best wording score
best_wording_index = all_wording_scores.index(max(all_wording_scores))

# Print the predictions from the best models
print("Best content score prediction:", all_content_scores[best_content_index])
print("Best wording score prediction:", all_wording_scores[best_wording_index])

def convert_to_positive(score):
  """
  Converts a score to a positive value between 0 and 1.

  Args:
    score: The score to convert.

  Returns:
    The converted score.
  """
  return 1 / (1 + np.exp(-score))

# Convert content and wording scores to positive values
positive_content_scores = [convert_to_positive(score) for score in all_content_scores]
positive_wording_scores = [convert_to_positive(score) for score in all_wording_scores]

# Print the converted scores
print("Positive content scores:", max(positive_content_scores))
print("Positive wording scores:", max(positive_wording_scores))

# Best content score prediction: -0.7367377877235413
# Best wording score prediction: -0.8119905591011047

#  text = 'Good Morning and my name is ali'
#normal
# Positive content scores: 0.3237179112624106
# Positive wording scores: 0.3074664819049631
#adhd
# Positive content scores: 0.5747161301908408
# Positive wording scores: 0.6091395074744388


Best content score prediction: 0.3011193573474884
Best wording score prediction: 0.44369661808013916
Positive content scores: 0.5747161301908408
Positive wording scores: 0.6091395074744388


In [ ]:
# prompt: convert content and wording score into positive

def convert_to_positive(score):
  """
  Converts a score to a positive value between 0 and 1.

  Args:
    score: The score to convert.

  Returns:
    The converted score.
  """
  return 1 / (1 + np.exp(-score))

# Convert content and wording scores to positive values
positive_content_scores = [convert_to_positive(score) for score in all_content_scores]
positive_wording_scores = [convert_to_positive(score) for score in all_wording_scores]

# Print the converted scores
print("Positive content scores:", max(positive_content_scores))
print("Positive wording scores:", max(positive_wording_scores))


Positive content scores: 0.3237179112624106
Positive wording scores: 0.3074664819049631
